In [1]:
import input
import string
import numpy as np

data = np.array([list(x) for x in input.read_input(12, True).splitlines()])
weights = np.zeros(data.shape, dtype=int)

start_pos = np.argwhere(data == 'S')[0]
start_pos = (start_pos[0], start_pos[1])

end_pos = np.argwhere(data == 'E')[0]
end_pos = (end_pos[0], end_pos[1])

for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        if data[y, x] != 'S' and data[y, x] != 'E':
            weights[y, x] = string.ascii_lowercase.index(data[y, x])
            
weights[start_pos[0], start_pos[1]] = 0
weights[end_pos[0], end_pos[1]] = string.ascii_lowercase.index('z')

weights

array([[ 0,  0,  1, 16, 15, 14, 13, 12],
       [ 0,  1,  2, 17, 24, 23, 23, 11],
       [ 0,  2,  2, 18, 25, 25, 23, 10],
       [ 0,  2,  2, 19, 20, 21, 22,  9],
       [ 0,  1,  3,  4,  5,  6,  7,  8]])

In [2]:
def get_neighbors(data, pos):
    if pos[0] + 1 < data.shape[0]:
        yield (pos[0] + 1, pos[1])
    if pos[0] - 1 >= 0:
        yield (pos[0] - 1, pos[1])
    if pos[1] + 1 < data.shape[1]:
        yield (pos[0], pos[1] + 1)
    if pos[1] - 1 >= 0:
        yield (pos[0], pos[1] - 1)
        
def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path = [current, *total_path]
    return total_path
        
def get_fastest_path(data, start_pos, end_pos, h):
    visited = set()
    queue = [start_pos]
    
    came_from = {}
    
    g_score = {start_pos: 0}
    f_score = {start_pos: h(start_pos)}
    
    while queue:
        pos = queue.pop(0)
        if pos in visited:
            continue
        
        visited.add(pos)
        
        if pos == end_pos:
            return reconstruct_path(came_from, pos)
        
        for neighbor in get_neighbors(data, pos):
            tentative_g_score = g_score[pos] + 1
            
            current_height = data[pos]
            neighbor_height = data[neighbor]

            if tentative_g_score < g_score.get(neighbor, float('inf')) and current_height + 1 >= neighbor_height:
                came_from[neighbor] = pos
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + h(neighbor)
                
                if neighbor not in queue:
                    queue.append(neighbor)

    return None

def get_manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

In [3]:
path = get_fastest_path(weights, start_pos, end_pos, lambda x: get_manhattan_distance(x, end_pos))

grid = np.zeros(weights.shape, dtype=str)
grid[start_pos[0], start_pos[1]] = 'S'
grid[end_pos[0], end_pos[1]] = 'E'

for y in range(weights.shape[0]):
    for x in range(weights.shape[1]):
        print('. ' if (y, x) not in path else f'{path.index((y, x)):02}', end=' ')
    print()

00 .  .  19 18 17 16 15 
01 02 .  20 29 28 27 14 
.  03 .  21 30 31 26 13 
.  04 05 22 23 24 25 12 
.  .  06 07 08 09 10 11 


In [4]:
min_path = np.Inf
for y in range(weights.shape[0]):
    for x in range(weights.shape[1]):
        if weights[y, x] == 0:
            path = get_fastest_path(weights, (y, x), end_pos, lambda x: get_manhattan_distance(x, end_pos))
            if (path is not None and len(path) - 1 < min_path) or min_path is None:
                min_path = len(path) - 1
print(min_path)

29
